In [1]:
from IPython.core.debugger import set_trace

#%load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="dark_background")

In [2]:
SENTIMENT_LABELLED_DATA_FILEPATH = '..\Data\Sentiment Analysis Dataset.csv'
DATA_PREPROCESSING_FILEPATH = '../Data Preprocessed'

TOKENIZER_OUTPUT_FILEPATH = 'tokenizer_100K.pickle'
MODEL_OUTPUT_FILEPATH = 'model_100K_glove'
# MODEL_OUTPUT_FILEPATH = 'model_100K_glove'
MODEL_OUTPUT_FILEPATH = 'model_100K_glove_91'

Import Dataset

In [3]:
txt_data = pd.read_csv(SENTIMENT_LABELLED_DATA_FILEPATH , sep='|', names=['col1'])

#split data into columns with ','
txt_data = txt_data.col1.str.split(',',  3, expand=True)
txt_data.columns = list(txt_data.iloc[0])
txt_data = txt_data.drop(0)
txt_data.index = np.subtract(txt_data.index, 1)
# x = txt_data.groupby('Sentiment')
# l=[x.get_group(i)['SentimentText'] for i in x.groups]


In [4]:
dataset = pd.concat([txt_data['SentimentText'], txt_data['Sentiment']], axis = 1)
dataset.columns = ['text', 'target']

In [22]:

print(dataset.info(verbose=True))
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1577838 entries, 0 to 1577837
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1577838 non-null  object
 1   target  1577838 non-null  object
dtypes: object(2)
memory usage: 36.1+ MB
None


,text,target
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0


In [6]:
#suffle
dataset = dataset.sample(frac = 1)
dataset

,text,target
1228040,Sometimes things do work in my favor,1
600218,and i fail epicly,0
1204630,This time I won't get back on twitter until th...,1
178857,"""@gruuvebot It's the damn back thing, otherwis...",1
209766,@jamescollier Friggan jealous! I need some mor...,0
...,...,...
842311,@Icepartyscott thats cool suzi is doin the ice...,0
848506,@IndywoodFILMS fly me to the UK and I will be ...,1
801538,"""has decided not to buy boxing - only my first...",0
803669,@galindafied Thanks! I'm a little rusty (have...,1


In [27]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append(DATA_PREPROCESSING_FILEPATH)
from data_preprocess import *

import pickle

In [28]:
text = NLP_preprocess(dataset)
text.preprocess_data()

text.set_tokenizer()
import pickle

# saving
with open(TOKENIZER_OUTPUT_FILEPATH, 'wb') as handle:
    pickle.dump(text.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

KeyboardInterrupt: 

In [9]:
train_dataset = dataset[:100000]
#train_dataset = dataset[:1000000]
train_labels = train_dataset.target.astype(float)
train_padded = text.tokenize_and_pad(train_dataset, train = True)
print(train_padded)
# saving
with open(TOKENIZER_OUTPUT_FILEPATH, 'wb') as handle:
    pickle.dump(text.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

[[   506    137     12 ...      0      0      0]
 [   463  27881      0 ...      0      0      0]
 [    15    109      5 ...      0      0      0]
 ...
 [   115     65      4 ...      0      0      0]
 [107090    541    210 ...      0      0      0]
 [107091  10846   5105 ...      0      0      0]]


In [10]:
text.set_embedding_matrix()

In [26]:
pd.DataFrame(text.embedding_matrix).info


NameError: name 'text' is not defined

In [12]:
test_dataset = dataset[:20000]
test_labels = test_dataset.target.astype(float)
test_padded = text.tokenize_and_pad(test_dataset, train = False)
print(test_padded)

[[  506   137    12 ...     0     0     0]
 [  463 27881     0 ...     0     0     0]
 [   15   109     5 ...     0     0     0]
 ...
 [21242    49  7475 ...     0     0     0]
 [43679    91  1129 ...     0     0     0]
 [  968    27  3499 ...     0     0     0]]


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

model = Sequential()

# trainable = False
#bidirectional

#Glove
model.add(
    Embedding(
        text.num_words,
        100,
        embeddings_initializer = Constant(text.embedding_matrix),
        input_length = text.max_length,
        trainable = False,
    )
)

#basic
# for i in range(0,200):
# #model.add(Embedding(text.num_words, 8, input_length=text.max_length))
#     model.add(Dense(1000, activation='relu'))
model.add(LSTM(100, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))

optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           10709300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_201 (Dense)            (None, 1)                 101       
Total params: 10,789,801
Trainable params: 80,501
Non-trainable params: 10,709,300
_________________________________________________________________


In [19]:
history = model.fit(
    train_padded, train_labels, epochs=40, validation_data=(test_padded, test_labels),
)

Epoch 1/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.3885 - accuracy: 0.8206 - val_loss: 0.3579 - val_accuracy: 0.8372
Epoch 2/40
3125/3125 [==============================] - 14s 5ms/step - loss: 0.3848 - accuracy: 0.8239 - val_loss: 0.3458 - val_accuracy: 0.8438
Epoch 3/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.3799 - accuracy: 0.8267 - val_loss: 0.3479 - val_accuracy: 0.8446
Epoch 4/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.3757 - accuracy: 0.8283 - val_loss: 0.3435 - val_accuracy: 0.8477
Epoch 5/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.3708 - accuracy: 0.8316 - val_loss: 0.3332 - val_accuracy: 0.8547
Epoch 6/40
3125/3125 [==============================] - 14s 5ms/step - loss: 0.3667 - accuracy: 0.8330 - val_loss: 0.3265 - val_accuracy: 0.8585
Epoch 7/40
3125/3125 [==============================] - 14s 4ms/step - loss: 0.3635 - accuracy: 0.8344 - val_loss: 0.3227 - val_ac

In [22]:
model.save(MODEL_OUTPUT_FILEPATH)

INFO:tensorflow:Assets written to: model_100K_glove_91\assets


INFO:tensorflow:Assets written to: model_100K_glove_91\assets


In [19]:
import random
#test_neg = random.choice(neg.values)
i = random.choice(test_dataset.index)
if test_dataset.loc[i]['target'] == '0':
    print('negative: ', test_dataset.loc[i]['text'])
else:
    print('positive: ', test_dataset.loc[i]['text'])
test_sequences = text.tokenizer.texts_to_sequences([test_dataset.loc[i]['text']])
test_padded2 = pad_sequences(
    test_sequences, maxlen=text.max_length, padding="post", truncating="post"
)
model.predict(test_padded2)

negative:  ohmymandy lucky met


array([[0.8493518]], dtype=float32)

In [31]:
# import keras
# model = keras.models.load_model('C:/Users/Konst/Desktop/PythonNotebooks')

In [ ]:
# model.summary()

In [ ]:
#model.save('')